# Project Overview

Brief description of the brain tumor image classification project.

---

# Data Preprocessing

Steps to preprocess the images (e.g., resizing, normalization, augmentation).

### Pasos recomendados

1. **Revisión y curado del dataset**
   - Eliminar duplicados, imágenes corruptas o mal etiquetadas.
   - Verificar coherencia entre nombres de archivos y carpetas.
   - Etiquetar por paciente si es posible (para evitar data leakage).

2. **División estratificada del dataset**
   - Separar en conjuntos: entrenamiento, validación y test.
   - Mantener proporción similar de clases en cada conjunto.
   - Asegurar que un mismo paciente no esté en más de un conjunto.

3. **Redimensionado y transformación de formato**
   - Unificar tamaño de imágenes (por ejemplo, 224x224 px).
   - Convertir a formato RGB si se va a usar un modelo preentrenado.
   - Cambiar a tipo de dato `float32`.

4. **Normalización de los valores de píxeles**
   - Escalar los valores de los píxeles a un rango como `[0, 1]` o `[-1, 1]`.
   - En imágenes médicas, aplicar normalización tipo Z-score si procede.

5. **Balanceo de clases**
   - Verificar si hay desbalance entre "tumor" y "sano".
   - Aplicar estrategias como oversampling, class weighting o pérdida focal.

6. **Data augmentation**
   - Diseñar transformaciones que no alteren la validez clínica.
   - Incluir rotaciones leves, zoom, flips horizontales, brillo, contraste...
   - Evitar transformaciones que inviertan lateralidad o distorsionen la anatomía.

7. **Preprocesado adaptado al modelo**
   - Aplicar la función de preprocesamiento correspondiente si se usa un modelo preentrenado (e.g., `preprocess_input` de ResNet o EfficientNet).


---

# Exploratory Data Analysis

Visualizations and statistics to understand the dataset.


### Pasos del análisis exploratorio

1. **Resumen general del dataset**
   - Número total de imágenes.
   - Número de imágenes por clase: `tumor` vs `sano`.
   - Número de imágenes por conjunto: entrenamiento, validación y test.
   - Verificar si las clases están balanceadas.

2. **Visualización de ejemplos por clase**
   - Mostrar imágenes representativas de cada clase.
   - Comprobar variabilidad visual entre las clases.
   - Identificar posibles imágenes mal clasificadas, oscuras o ruidosas.

3. **Distribución de tamaños y resoluciones**
   - Analizar si todas las imágenes tienen dimensiones coherentes.
   - Detectar outliers en tamaño o relación de aspecto.

4. **Histograma de intensidades de píxeles**
   - Visualizar histogramas para imágenes individuales o agrupadas por clase.
   - Detectar diferencias de brillo o contraste entre clases.

5. **Comprobación de duplicados o imágenes muy similares**
   - Verificar si hay imágenes repetidas (por nombre o por contenido).
   - Usar técnicas básicas como hashing, distancia estructural, etc.

---

# Model Building

Define the architecture of the classification model (e.g., CNN).

# Model Training

Train the model using the training dataset.

# Model Evaluation

Evaluate the model performance on the validation/test dataset.

# Predictions

Make predictions on new or unseen images.

# Conclusions and Next Steps

Summarize findings and suggest future improvements.